In [102]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time

In [25]:
# 웹 드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
driver.get("https://www.siksinhot.com/")

In [41]:
search_list = [
    "강동구","강서구","강남구","도봉구","양천구","마포구","서대문구","은평구","서초구","구로구","중구",
    "동대문구","중랑구","광진구","송파구","강북구","관악구"
]

In [42]:
driver.find_element(By.CSS_SELECTOR,"#header > header > div.searchBox > div").click()

In [47]:
driver.find_element(By.CSS_SELECTOR,"#searchInput").send_keys("강북구")
driver.find_element(By.CSS_SELECTOR,"#searchInput").send_keys(Keys.RETURN)

In [52]:
for search in search_list:
    input_box = driver.find_element(By.CSS_SELECTOR,"#header > header > div.searchBox > div")
    input_box.click()
    time.sleep(4)
    
    search_input = driver.find_element(By.CSS_SELECTOR,"#searchInput")
    search_input.send_keys(search)
    search_input.send_keys(Keys.RETURN)
    time.sleep(4)
    try:
        while True:
            more_box = driver.find_element(By.CSS_SELECTOR, "#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > a")

            more_box.click()
            time.sleep(2)  # 클릭 후 로딩을 위한 대기 시간 (필요에 따라 조절)

    except:
        page_source = driver.page_source
        all_pages.append(page_source)
        pass 
    
    

KeyboardInterrupt: 

In [61]:
len(all_pages)

14

In [ ]:
for page in all_pages:
    soup = bs(page,"lxml")
    soup.select()

In [69]:
url = "https://www.siksinhot.com/search?keywords=%EC%84%9C%EB%8C%80%EB%AC%B8%EA%B5%AC"
r = requests.get(url)
soup = bs(r.text,"lxml")


In [73]:
main = soup.select("#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > ul > li")

In [104]:
page_list = []

for page in all_pages:
    soup = bs(page,"lxml")
    main = soup.select("#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > ul > li")
    for i in main:
        dict_list = {}
        title = i.select_one("h2").text
        location = i.select_one(".cate>a:first-child").text
        division = i.select_one(".cate>a:last-child").text
        view = i.select_one(".counting").text
        score = i.select_one(".score").text
        book_mark = i.select_one(".counting>span:nth-of-type(2)").text
        print()
        dict_list["제목"] = title 
        dict_list["지역"] = location
        dict_list["분류"] = division
        dict_list["뷰"] = view
        dict_list["평점"] = score
        dict_list["북마크"] = book_mark
        page_list.append(dict_list)

In [105]:
page_list

[{'제목': '서울신라호텔팔선',
  '지역': '동대문/장충/신당',
  '분류': '중식',
  '뷰': '31056103',
  '평점': '4.3',
  '북마크': '103'},
 {'제목': '피에르가니에르서울 롯데호텔서울',
  '지역': '시청',
  '분류': '프렌치',
  '뷰': '33466139',
  '평점': '4.2',
  '북마크': '139'},
 {'제목': '라연 서울신라호텔',
  '지역': '동대문/장충/신당',
  '분류': '한정식',
  '뷰': '2906594',
  '평점': '4.1',
  '북마크': '94'},
 {'제목': '서촌계단집',
  '지역': '서촌누하동/청운효자동',
  '분류': '해물탕/해물요리',
  '뷰': '45706227',
  '평점': '4.0',
  '북마크': '227'},
 {'제목': 'STAY 시그니엘서울',
  '지역': '잠실-롯데월드/롯데타워',
  '분류': '프렌치',
  '뷰': '5177744',
  '평점': '3.7',
  '북마크': '44'},
 {'제목': '모수서울',
  '지역': '이태원-한남동/독서당로',
  '분류': '유러피안',
  '뷰': '957812',
  '평점': '3.5',
  '북마크': '12'},
 {'제목': '서산꽃게',
  '지역': '마포/공덕',
  '분류': '꽃게/랍스터/새우',
  '뷰': '50517118',
  '평점': '4.3',
  '북마크': '118'},
 {'제목': '서관면옥',
  '지역': '교대/서초/남부터미널',
  '분류': '냉면/막국수',
  '뷰': '1841520',
  '평점': '4.2',
  '북마크': '20'},
 {'제목': '스테이크하우스 그랜드하얏트서울',
  '지역': '이태원-한남동/독서당로',
  '분류': '스테이크',
  '뷰': '166927',
  '평점': '4.2',
  '북마크': '7'},
 {'제목': '품 서울',
  '지역': '삼성동

In [106]:
pd.DataFrame(page_list)

,제목,지역,분류,뷰,평점,북마크
0,서울신라호텔팔선,동대문/장충/신당,중식,31056103,4.3,103
1,피에르가니에르서울 롯데호텔서울,시청,프렌치,33466139,4.2,139
2,라연 서울신라호텔,동대문/장충/신당,한정식,2906594,4.1,94
3,서촌계단집,서촌누하동/청운효자동,해물탕/해물요리,45706227,4.0,227
4,STAY 시그니엘서울,잠실-롯데월드/롯데타워,프렌치,5177744,3.7,44
...,...,...,...,...,...,...
1670,서울우진갈비,서대문/충정로,돼지갈비,20101,평가중,1
1671,서부족발,서대문/충정로,족발/보쌈,1618,평가중,
1672,아빠곰탕 서대문역점,서대문/충정로,설렁탕/곰탕/갈비탕,35030,평가중,0
1673,서대문골목 서대포집,서대문/충정로,돼지구이,2337,평가중,
